In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

#import data
data = pd.read_csv('data.csv')
data.head()

data["gender"].value_counts()

data["age"].value_counts()

data["age"]

modified_values = data[data.age<151].age
print(modified_values.mean())
print(modified_values.median())
print(modified_values.mode()[0])

#checking for missing values
x = data.isnull().sum()
set(x)

data.age[data.age < 150].hist()

#errorvalues = data[data.age > 150].age
#print(errorvalues)    
#data.replace(to_replace = errorvalues, value = 22)

# Create X (all the feature columns)
X = data.drop(["affiliative", 
              "selfenhancing",
              "agressive",
              "selfdefeating",
              "age"],
              axis=1)

# Create y (the target column)
y = data[["gender"]]

X

y
#y = label_binarize(y, classes=[0, 1, 2])
#n_classes = y.shape[1]


#split into train and test values
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

#predict using random forest multiclass classifier
from sklearn.ensemble import RandomForestClassifier

# Setup random seed
np.random.seed(42)
# Instantiate and fit the model (on the training set)
RFC = RandomForestClassifier()
RFC.fit(X_train, y_train.values.ravel())

# Check the score of the model (on the test set)
RFC.score(X_test, y_test)
print(f"Random Forest Classifier Accuracy: {RFC.score(X_test, y_test) * 100:.2f}%")

from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)
clf = DecisionTreeClassifier(max_depth = 1)
clf.fit(X_train, y_train.values.ravel())


# Check the score of the model (on the test set)
clf.score(X_test, y_test)
print(f"Decision Tree Accuracy: {clf.score(X_test, y_test) * 100:.2f}%")

from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(X_train, y_train.values.ravel())


# Check the score of the model (on the test set)
neigh.score(X_test, y_test)
print(f"K nearest Neightbour Classifier Accuracy: {neigh.score(X_test, y_test) * 100:.2f}%")

#Confusion matrix
#for random forest
from sklearn.metrics import confusion_matrix

y_preds = RFC.predict(X_test)

confusion_matrix(y_test, y_preds)


from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(RFC, X, y)

#for Decision Trees
from sklearn.metrics import confusion_matrix

y_preds = clf.predict(X_test)

confusion_matrix(y_test, y_preds)


from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(clf, X, y)

#for K Nearest Neighbours
from sklearn.metrics import confusion_matrix

y_preds = neigh.predict(X_test)

confusion_matrix(y_test, y_preds)

from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(neigh, X, y)

#Classification report for precision and recall
#Random Forest
from sklearn.metrics import classification_report
y_preds_RFC = RFC.predict(X_test)
print(classification_report(y_test, y_preds_RFC))

#Decision Trees
from sklearn.metrics import classification_report
y_preds_clf = clf.predict(X_test)
print(classification_report(y_test, y_preds_clf))

#K Nearest Neighbours
from sklearn.metrics import classification_report
y_preds_neigh = neigh.predict(X_test)
print(classification_report(y_test, y_preds_neigh))

y_preds_proba_RFC = RFC.predict_proba(X_test)

y_preds_proba_RFC

roc_auc_score(y_test,y_preds_proba_RFC,multi_class="ovr")

y_preds_proba_clf = clf.predict_proba(X_test)

y_preds_proba_clf

roc_auc_score(y_test,y_preds_proba_clf,multi_class="ovr")

y_preds_proba_neigh = neigh.predict_proba(X_test)

y_preds_proba_neigh

roc_auc_score(y_test,y_preds_proba_neigh,multi_class="ovr")

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_preds_proba_RFC[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='-.',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='-.',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='-.',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_preds_proba_clf[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

# roc curve for classes
fpr = {}
tpr = {}
thresh ={}

n_class = 3

for i in range(n_class):    
    fpr[i], tpr[i], thresh[i] = roc_curve(y_test, y_preds_proba_neigh[:,i], pos_label=i)
    
# plotting    
plt.plot(fpr[0], tpr[0], linestyle='--',color='orange', label='Class 0 vs Rest')
plt.plot(fpr[1], tpr[1], linestyle='--',color='green', label='Class 1 vs Rest')
plt.plot(fpr[2], tpr[2], linestyle='--',color='blue', label='Class 2 vs Rest')
plt.title('Multiclass ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive rate')
plt.legend(loc='best')
plt.savefig('Multiclass ROC',dpi=300);    

